In [11]:
# Essential libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import random

from collections import Counter
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Activation, Dropout, Bidirectional, LSTM
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder

In [12]:
# Importing training and test data
sample = pd.read_csv('sample.csv')
test = pd.read_csv('test.csv', header=None, index_col=0)
data = pd.read_csv('train.csv', header=None, index_col=0)
df_train = data.drop(columns=[961])
y_train = data[961]

In [13]:
# Oversampling minority classes
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(df_train, y_train)

In [14]:
def scale_data(df):
    scaler = RobustScaler()
    scaler = scaler.fit(X_res)
    df_scaled  = scaler.transform(df.to_numpy())

    return df_scaled

In [15]:
# Scaling train and test data
df_train = scale_data(X_res)
df_test = scale_data(test)

In [16]:
# Reshaping training and test data
X_train = df_train.reshape(df_train.shape[0], 16, 60)
X_test = df_test.reshape(df_test.shape[0], 16, 60)

In [17]:
y_train = np.array(y_res)
y_train = y_train.reshape(len(y_train), 1)

In [18]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
# one hot encoding the labels
enc = enc.fit(y_train)

matrix_Y = enc.transform(y_train)

In [19]:
# Spliting training and validation data
X_train, x_test, y_train, y_test = train_test_split(X_train, matrix_Y, test_size=0.3, random_state=42, stratify=matrix_Y)

In [20]:
# Single layer Bi-LSTM model
model = keras.Sequential()
model.add(
    Bidirectional(
      LSTM(
          units=128, 
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [23]:
# Fitting the model
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_data=(x_test,y_test),
    shuffle=True
)

Epoch 1/200
317/317 [==============================] - 12s 9ms/step - loss: 3.2953 - acc: 0.1956 - val_loss: 1.8885 - val_acc: 0.5171
Epoch 2/200
317/317 [==============================] - 2s 7ms/step - loss: 1.8126 - acc: 0.5217 - val_loss: 1.3417 - val_acc: 0.6489
Epoch 3/200
317/317 [==============================] - 2s 7ms/step - loss: 1.3298 - acc: 0.6467 - val_loss: 1.0759 - val_acc: 0.7160
Epoch 4/200
317/317 [==============================] - 2s 7ms/step - loss: 1.0711 - acc: 0.7045 - val_loss: 0.9476 - val_acc: 0.7403
Epoch 5/200
317/317 [==============================] - 2s 7ms/step - loss: 0.9138 - acc: 0.7410 - val_loss: 0.8701 - val_acc: 0.7605
Epoch 6/200
317/317 [==============================] - 2s 7ms/step - loss: 0.7998 - acc: 0.7681 - val_loss: 0.8077 - val_acc: 0.7790
Epoch 7/200
317/317 [==============================] - 2s 7ms/step - loss: 0.7017 - acc: 0.7977 - val_loss: 0.7654 - val_acc: 0.7905
Epoch 8/200
317/317 [==============================] - 2s 7ms/step -

In [24]:
# Predicting test data
y_pred = model.predict(X_test)
y_pred=enc.inverse_transform(y_pred)

In [25]:
# Creating csv file like sample.csv
submission=pd.DataFrame(columns=sample.columns)
submission['Id']=sample['Id']
submission['Category']=pd.DataFrame(y_pred)[0]
submission.to_csv("baseline.csv", index=False)